#### 이 실습은 [링크](https://wiserloner.tistory.com/1049)의 자료를 참고하여 구성하였습니다.

### 필요한 라이브러리를 불러옵니다.

In [ ]:
!pip install tensorflow_datasets --upgrade
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
tfds.disable_progress_bar()
AUTOTUNE = tf.data.experimental.AUTOTUNE

### 본 실기에서는 TensorFlow에서 제공하는 데이터셋 중 콩잎 데이터셋을 이용합니다.

### 콩잎 데이터셋은 세 가지 상태의 콩잎을 클래스로 가지는 데이터셋입니다.

### 세 가지 상태는 Angular Leaf Spot(0), Bean Rust(1), Healthy(2)로, 실기에서는 세 상태를 분류하는 이미지 task를 수행하겠습니다.

### 먼저 데이터셋을 불러오겠습니다.

In [ ]:
'''데이터셋 로드를 위해 tfds.load 를 사용합니다. 해당 데이터셋을 train과 validation으로 split합니다.'''
(train_set, val_set, test_set), info =  tfds.load(
    'Beans',
    split=('train','validation', 'test'),
    as_supervised=True, 
    with_info=True,
)

In [ ]:
'''데이터 포인트 개수를 확인합니다.'''
num_train = info.splits['train'].num_examples      # 학습 데이터 수
num_val = info.splits['validation'].num_examples   # 검증 데이터 수
num_test = info.splits['test'].num_examples        # 테스트 데이터 수
print('학습 데이터 수 = %s\n검증 데이터 수 = %s\n테스트 데이터 수 = %s' %(num_train, num_val, num_test))

### 데이터가 잘 받아졌는지 샘플 데이터를 확인해봅시다.

In [ ]:
get_label_name = info.features['label'].int2str

for image, label in train_set.take(3):
    plt.figure()
    plt.imshow(image)
    plt.title(get_label_name(label))

### 이미지 데이터가 normalize되어 있는지 확인합니다.

In [ ]:
''' 학습 데이터셋에서 샘플 하나를 살펴봅니다.'''
for image, label in train_set.take(1):
    pass

print('이미지 shape = %s' %(image.shape))
print('이미지 픽셀 최대값 = %s\n이미지 픽셀 최소값 = %s' %(tf.math.reduce_max(image), tf.math.reduce_min(image)))
print('이미지 라벨 = %s' %(label))

### 라벨값은 0,1,2 세 값을 가집니다. 

### 픽셀 값이 0~255의 범위이므로 normalize 함수를 정의합니다. 

In [ ]:
'''전처리 함수를 정의합니다.'''
def convert(image, label):
    image = tf.cast(image, tf.float32)
    image = image/255.0
    # image = tf.image.resize(image, (500,500))   # 필요하다면 모델 입력 shape에 맞게 resize를 진행할 수 있습니다. 
    return image, label

### 데이터셋 객체를 이용해 학습 데이터를 준비해보겠습니다.

In [ ]:
BATCH_SIZE = 32

train_batches = (
    train_set
    .shuffle(num_train)
    .map(convert, num_parallel_calls=AUTOTUNE)  # map을 통해 위에서 정의한 전처리 함수를 콜백함수로 적용시킵니다.
    .batch(BATCH_SIZE)  # 배치 사이즈를 설정합니다. 
    .prefetch(AUTOTUNE)
) 

### 같은 방식으로 검증 데이터와 테스트 데이터를 준비합니다.

In [ ]:
val_batches = (
    val_set
    .map(convert, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
)

test_batches = (
    test_set
    .map(convert, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
)

### 이제 ImageNet 데이터로 사전학습된 VGG16 모델을 이용해 task에 맞게 레이어를 추가해보겠습니다. 

### 모델 로드 시 include_top을 False로 지정하면 Flatten 레이어와 FC Classifier 블록을 제외하고 받을 수 있습니다.

### 모델을 로드하고 Flatten 레이어까지 추가합니다. 

In [ ]:
IMG_SHAPE = (500,500,3)

'''사전 훈련된 VGG16 모델로부터 기본 모델을 만듭니다.'''
vgg_pre = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                      include_top=False,
                                      weights='imagenet')    # ImageNet pre-trained weights를 로드합니다. 
# vgg_pre 모델의 구조를 vgg_pre.summary()로 확인하고 강의 자료의 그림과 비교해보세요.
'''Flatten 레이어를 추가합니다.'''
output = vgg_pre.layers[-1].output
output = tf.keras.layers.Flatten()(output)
vgg1 = Model(vgg_pre.input, output)

### 위에서 정의한 vgg1 모델을 Feature Extractor로 사용할 것이므로, Feature Extractor 부분을 freeze하고 결과를 확인합니다.

In [ ]:
vgg1.trainable = False
layers = [(layer, layer.name, layer.trainable) for layer in vgg1.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])   

### 모든 레이어가 잘 freeze된 것을 볼 수 있습니다. 
### 이제 vgg1 모델에 fully-connected layer를 추가하여 모델이 세 개의 상태를 분류할 수 있게 만들어 줍시다.

In [ ]:
tf.random.set_seed(2021)
model1 = Sequential()
model1.add(vgg1)
model1.add(Dense(128, activation='relu'))
model1.add(Dense(128, activation='relu'))
model1.add(Dense(3))    # 세 개의 상태를 분류하는 task입니다.

### 모델이 잘 작동할지 학습해 보겠습니다. 

In [ ]:
model1.compile(optimizer = optimizers.Adagrad(learning_rate=0.001),
               loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
               metrics='accuracy')
history = model1.fit(train_batches, epochs=5, validation_data=val_batches)

### 5 epoch을 학습한 결과로는 꽤 괜찮은 것 같습니다. 우리는 그저 누군가가 미리 학습해 놓은 결과를 가지고 와서 간단한 모델을 만들었을 뿐입니다. 

### 하지만 사전학습된 weights를 사용하지 않고 random하게 초기화된 값에서 모델 학습을 시작한다면 어떨까요? 

### 사전학습된 weights가 정말로 효과가 있는지 확인해 봅시다.
### 동일한 모델이지만 weights만 다른 VGG16 모델을 로드합니다. 

In [ ]:
vgg_random = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                         include_top=False,
                                         weights=None)     # random하게 초기화된 값으로 로드합니다. 
output = vgg_random.layers[-1].output
output = tf.keras.layers.Flatten()(output)
vgg2 = Model(vgg_random.input, output)

### Random하게 초기화된 weights이므로 weights를 freeze하면 안될 것입니다. 레이어 weights가 업데이트되는 상태인지 확인해 봅시다. 

In [ ]:
layers = [(layer, layer.name, layer.trainable) for layer in vgg2.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

### 좋습니다. 이제 위와 동일한 방법으로 모델을 만들어 학습해 보겠습니다. 

### (학습이 10분 정도 걸립니다... 학습 시작 후 잠시 휴식하겠습니다.

In [ ]:
tf.random.set_seed(2021)
model2 = Sequential()
model2.add(vgg2)
model2.add(Dense(128, activation='relu'))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(3))

model2.compile(optimizer = optimizers.Adagrad(learning_rate=0.001),    # learning rate을 조금 높였습니다. 
               loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
               metrics='accuracy')
aug_history = model2.fit(train_batches, epochs=5, validation_data=val_batches)

### 사전학습된 weights의 효과는 명백해 보입니다! Random한 값으로 초기화된 모델로 바닥에서부터 학습하면 학습 시간도 오래 걸리고 정확도도 높지 않습니다. 

### 사전학습된 weights로 학습을 시작하면, 유용한 feature를 가지고 시작하므로 높은 정확도로 빠르게 수렴합니다. 

### 하지만 사전학습된 weights, 즉 ImageNet으로 학습된 feature는 해당 task에 좀 더 적합할 것이고, 콩잎의 상태를 분류하기 위해서는 학습된 feature를 수정해도 좋을 것 같습니다. 

### VGG16 모델의 block5를 fine-tuning하고 결과를 비교해보겠습니다. 

In [ ]:
'''ImageNet 사전학습된 weights를 불러옵니다.'''
vgg_pre = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                      include_top=False,
                                      weights='imagenet')
output = vgg_pre.layers[-1].output
output = tf.keras.layers.Flatten()(output)
vgg3 = Model(vgg_pre.input, output)

### VGG16 모델의 Layer Name을 block1부터 block5까지 순서대로이므로, block5_conv1 레이어부터 freeze를 풀겠습니다. 

In [ ]:
vgg3.trainable = False

set_trainable = False
for layer in vgg3.layers:
    if layer.name in ['block5_conv1']:
        set_trainable = True
    if set_trainable:
        layer.trainable = True

layers = [(layer, layer.name, layer.trainable) for layer in vgg3.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

### 이제 동일한 방법으로 학습을 진행해보겠습니다. 

In [ ]:
tf.random.set_seed(2021)
model3 = Sequential()
model3.add(vgg3)
model3.add(Dense(128, activation='relu'))
model3.add(Dense(128, activation='relu'))
model3.add(Dense(3))

model3.compile(optimizer = optimizers.Adagrad(learning_rate=0.001),
               loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
               metrics='accuracy')
aug_history = model3.fit(train_batches, epochs=5, validation_data=val_batches)

### 정확도에 개선이 있는 것을 확인할 수 있습니다.  

### 적절하게 fine-tuning을 이용하면 현재 task에 더욱 적합한 feature를 배우는 데에 도움을 줄 수 있습니다. 

### Fine-tuning을 거친 모델을 통해 test 이미지를 추론해보고 결과를 시각화하겠습니다. 

In [ ]:
# 새로운 모델로 추론
pred_batch = model3.predict(test_batches)
pred_label = np.argmax(pred_batch, axis=-1)

# 실제 라벨 (Ground Truth)
temp_test = tfds.as_numpy(test_set)
true_img = np.array([x[0] for x in temp_test])
true_label = np.array([x[1] for x in temp_test])

plt.figure(figsize=(20,16))
plt.subplots_adjust(hspace=0.3)
for n in range(20):
    plt.subplot(5,4,n+1)
    plt.imshow(true_img[n])
    color = "green" if pred_label[n] == true_label[n] else "red"
    plt.title(get_label_name(pred_label[n]).title(), color=color)
    plt.axis('off')
_ = plt.suptitle("Model predictions (green: correct, red: incorrect)")

### 초록색 표시는 모델이 맞춘 정답, 붉은색 표시는 모델이 틀린 경우입니다.

### 테스트 데이터에 대해 결과가 괜찮게 나온 것 같습니다. 